## Download Data from API

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = "http://www.rnd.tmd.go.th/hi_monitor/Jan2565-Mar2567/"
save_dir = r"D:\Gistda\P_GONE\TMD_API\XLSX"

# Get the HTML of the page
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

download_count = 0

for link in soup.find_all('a', href=True):
    file_name = link['href']
    if file_name.endswith('.xlsx'):
        file_url = url + file_name
        file_path = os.path.join(save_dir, file_name)
        
        # Download the file
        file_response = requests.get(file_url)
        if file_response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(file_response.content)
            download_count += 1
            # print(f"Downloaded: {file_name}")

print(f"Total files downloaded: {download_count}")


In [13]:
import pandas as pd

file_path = r"D:\Gistda\P_GONE\TMD_API\XLSX\48300_MAEHONGSON.xlsx"
data = pd.read_excel(file_path)
# print(data)
print("Headers:", data.columns.tolist())
# print(data.head(10))

Headers: ['Date', 'HI_2565_100', 'HI_2565_400', 'HI_2565_700', 'HI_2565_1000', 'HI_2565_1300', 'HI_2565_1600', 'HI_2565_1900', 'HI_2565_2200', 'HI_2566_100', 'HI_2566_400', 'HI_2566_700', 'HI_2566_1000', 'HI_2566_1300', 'HI_2566_1600', 'HI_2566_1900', 'HI_2566_2200', 'HI_2567_100', 'HI_2567_400', 'HI_2567_700', 'HI_2567_1000', 'HI_2567_1300', 'HI_2567_1600', 'HI_2567_1900', 'HI_2567_2200']


In [ ]:
import pandas as pd
import os

# ฟังก์ชันสำหรับอ่านไฟล์และแยกข้อมูลตามปี
def process_files(input_directory, output_directory):
    # สร้าง DataFrame เปล่าสำหรับเก็บข้อมูลแต่ละปี
    df_2565 = pd.DataFrame()
    df_2566 = pd.DataFrame()
    df_2567 = pd.DataFrame()

    # อ่านไฟล์ทั้งหมดใน input_directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(input_directory, filename)
            df = pd.read_excel(file_path)

            # ตัดส่วนของชื่อคอลัมน์ที่ไม่จำเป็น เช่น "HI_2565_100" → "100"
            new_columns = []
            for col in df.columns:
                # ตัด "HI_" และ "2565", "2566", "2567" ออก แล้วเก็บแค่ตัวเลขสุดท้ายหลัง "_"
                parts = col.split('_')
                if len(parts) > 2:
                    new_columns.append(parts[-1])  # เก็บแค่ตัวเลขหลังสุด
                else:
                    new_columns.append(col)  # ถ้าไม่ใช่ในรูปแบบที่ต้องการ ให้เก็บไว้ตามเดิม

            df.columns = new_columns  # อัปเดตชื่อคอลัมน์

            # แยกข้อมูลออกตามปี 2565, 2566, 2567
            df_2565_temp = df[['Date', '100', '400', '700', '1000', '1300', '1600', '1900', '2200']]
            df_2566_temp = df[['Date', '100', '400', '700', '1000', '1300', '1600', '1900', '2200']]
            df_2567_temp = df[['Date', '100', '400', '700', '1000', '1300', '1600', '1900', '2200']]

            # รวมข้อมูลทั้งหมดใน DataFrame ของแต่ละปี
            df_2565 = pd.concat([df_2565, df_2565_temp])
            df_2566 = pd.concat([df_2566, df_2566_temp])
            df_2567 = pd.concat([df_2567, df_2567_temp])

    # สร้างเส้นทางไฟล์ใหม่สำหรับบันทึก
    output_file = os.path.join(output_directory, 'combined_data.xlsx')

    # บันทึกข้อมูลในแต่ละปีเป็นไฟล์ Excel
    with pd.ExcelWriter(output_file) as writer:
        df_2565.to_excel(writer, sheet_name='2565', index=False)
        df_2566.to_excel(writer, sheet_name='2566', index=False)
        df_2567.to_excel(writer, sheet_name='2567', index=False)

# เรียกใช้ฟังก์ชัน
input_directory = r'D:\Gistda\P_GONE\TMD_API\XLSX'  # เส้นทางที่เก็บไฟล์ Excel
output_directory = r'D:\Gistda\P_GONE\TMD_API\test'  # เส้นทางที่ต้องการเก็บไฟล์ผลลัพธ์
process_files(input_directory, output_directory)


Processing completed.


## Split 2565 2566 2567 and cal Min Max and Mean

In [ ]:
import os
import pandas as pd

# input_path = r'D:\Gistda\P_GONE\TMD_API\test'
input_path = r'D:\Gistda\P_GONE\TMD_API\XLSX'
output_path_2565 = r'D:\Gistda\P_GONE\TMD_API\2565'
output_path_2566 = r'D:\Gistda\P_GONE\TMD_API\2566'
output_path_2567 = r'D:\Gistda\P_GONE\TMD_API\2567'

# ตรวจสอบว่าโฟลเดอร์ output มีอยู่หรือไม่ ถ้าไม่มีให้สร้างขึ้นมา
os.makedirs(output_path_2565, exist_ok=True)
os.makedirs(output_path_2566, exist_ok=True)
os.makedirs(output_path_2567, exist_ok=True)

# อ่านไฟล์ทั้งหมดใน directory
files = os.listdir(input_path)

# ฟังก์ชันสำหรับแยกคอลัมน์ตามปีและคำนวณค่า min, max, mean
def split_year_columns_and_calculate(df):
    # print("Columns in the file:", df.columns)
    
    # แยกคอลัมน์ตามปี
    df_2565 = df[['Date'] + [col for col in df.columns if '2565' in col]]
    df_2566 = df[['Date'] + [col for col in df.columns if '2566' in col]]
    df_2567 = df[['Date'] + [col for col in df.columns if '2567' in col]]
    
    # คำนวณค่า min, max, mean ในแต่ละแถว
    for year_df in [df_2565, df_2566, df_2567]:
        year_df['Min'] = year_df.drop(columns=['Date']).min(axis=1)  # หาค่า min ในแต่ละแถว
        year_df['Max'] = year_df.drop(columns=['Date']).max(axis=1)  # หาค่า max ในแต่ละแถว
        year_df['Mean'] = year_df.drop(columns=['Date']).mean(axis=1)  # หาค่า mean ในแต่ละแถว
    
    return df_2565, df_2566, df_2567

# อ่านและแยกไฟล์ Excel
for file in files:
    if file.endswith('.xlsx'):
        file_path = os.path.join(input_path, file)
        
        # อ่านไฟล์ Excel
        df = pd.read_excel(file_path)
        
        # แสดงชื่อคอลัมน์
        # print(f"Reading file: {file}")
        # print("Columns:", df.columns)
        
        # แยกข้อมูลตามปีและคำนวณค่า min, max, mean
        df_2565, df_2566, df_2567 = split_year_columns_and_calculate(df)
        
        # สร้างชื่อไฟล์ใหม่โดยแยกตามปี
        base_filename = os.path.splitext(file)[0]  # ใช้ชื่อไฟล์เดิมโดยไม่มีนามสกุล
        new_filename_2565 = f"STAT_{base_filename}_2565.xlsx"
        new_filename_2566 = f"STAT_{base_filename}_2566.xlsx"
        new_filename_2567 = f"STAT_{base_filename}_2567.xlsx"
        
        # บันทึกไฟล์ลงในแต่ละโฟลเดอร์
        df_2565.to_excel(os.path.join(output_path_2565, new_filename_2565), index=False)
        df_2566.to_excel(os.path.join(output_path_2566, new_filename_2566), index=False)
        df_2567.to_excel(os.path.join(output_path_2567, new_filename_2567), index=False)
        
        print(f"Processed: XLSX Stat")

print("Processing Completed.")
